Download Data

In [1]:
from utils.utils_cifar import get_train_data_loader, get_test_data_loader, imshow, classes

trainloader = get_train_data_loader("/tmp/pytorch-example/cifar-10-data")
testloader = get_test_data_loader("/tmp/pytorch-example/cifar-10-data")

1.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

4.7%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

7.8%IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

11.0%IOPub me

Extracting /tmp/pytorch-example/cifar-10-data/cifar-10-python.tar.gz to /tmp/pytorch-example/cifar-10-data
Files already downloaded and verified


In [2]:
# There should be the original tar file along with the extracted data directory. (cifar-10-python.tar.gz, cifar-10-batches-py)
! ls /tmp/pytorch-example/cifar-10-data

cifar-10-batches-py  cifar-10-python.tar.gz


Training

In [7]:
from sagemaker import get_execution_role
import boto3
try: 
    role = get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_role')['Role']['Arn']

Couldn't call 'get_role' to get Role ARN from role name sagemaker_local to get Role path.


arn:aws:iam::224863548990:role/sagemaker_role


In [12]:
# Lets set up our SageMaker notebook instance for local mode.
!/bin/bash ./utils/setup.sh

sudo: a password is required
The user does not have root access. Everything required to run the notebook is already installed and setup. We are good to go!


In [13]:
import os
import subprocess

instance_type = "local"

if subprocess.call("nvidia-smi") == 0:
    ## Set type to GPU if one is present
    instance_type = "local_gpu"

print("Instance type = " + instance_type)

Sun Mar 13 00:31:13 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.103.01   Driver Version: 470.103.01   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:01:00.0 Off |                  N/A |
| N/A   46C    P8    12W /  N/A |    702MiB /  7982MiB |     16%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [17]:
from sagemaker.estimator import Estimator

hyperparameters = {"epochs": 1}

estimator = Estimator(
    role=role,
    train_instance_count=1,
    train_instance_type=instance_type,
    image_uri="lidar:latest",
    hyperparameters=hyperparameters,
)

estimator.fit("file:///tmp/pytorch-example/cifar-10-data")

predictor = estimator.deploy(1, instance_type)

train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


Pulling algo-1-o6jw8 (lidar:latest)...
The image for the service you're trying to recreate has been removed. If you continue, volume data could be lost. Consider backing up your data before continuing.

Continue with the new image? [yN]pull access denied for lidar, repository does not exist or may require 'docker login': denied: requested access to the resource is denied


RuntimeError: Failed to run: ['docker-compose', '-f', '/tmp/tmpn_016avc/docker-compose.yaml', 'up', '--build', '--abort-on-container-exit'], Process exited with code: 1

Prediction

In [ ]:
import torchvision, torch
import numpy as np

from sagemaker.predictor import json_serializer, json_deserializer

# get some test images
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(images))
print("GroundTruth: ", " ".join("%4s" % classes[labels[j]] for j in range(4)))

# predictor.accept = "application/json"
# predictor.content_type = "application/json"

predictor.serializer = json_serializer
predictor.deserializer = json_deserializer

outputs = predictor.predict(images.numpy())

_, predicted = torch.max(torch.from_numpy(np.array(outputs)), 1)

print("Predicted: ", " ".join("%4s" % classes[predicted[j]] for j in range(4)))

In [ ]:
predictor.delete_endpoint()